# Multi-Class Classification

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import ipywidgets
from ipywidgets import interact, interactive, interact_manual
import IPython
import pylab
pylab.rcParams['figure.figsize'] = (16, 8)

from utilities import plot_helpers
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn import datasets
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier

In [ ]:
def build_confusion_matrix(pred_label, true_label, num_classes=2):
    """This works for predictions in {0, 1, ..., Num Classes}."""
    confusion_matrix = np.zeros((num_classes, num_classes))
    for row in range(num_classes):
        for col in range(num_classes):
            confusion_matrix[row, col] = np.sum(np.bitwise_and(pred_label == row, true_label == col))
    return confusion_matrix

def accuracy(confusion_matrix):
    return np.sum(np.diag(confusion_matrix)) / np.sum(confusion_matrix)

def precision(confusion_matrix):
    correct = np.diag(confusion_matrix)
    pred = np.sum(confusion_matrix, axis=1)
    pred[pred == 0] = 1  # to avoid nan.
    
    return np.min(correct / pred)

def recall(confusion_matrix):
    correct = np.diag(confusion_matrix)
    pred = np.sum(confusion_matrix, axis=0)
    pred[pred == 0] = 1  # to avoid nan.

    return np.min(correct / pred)

def f1_score(confusion_matrix):
    correct = np.diag(confusion_matrix)
    rec_dem = np.sum(confusion_matrix, axis=0)
    prec_dem = np.sum(confusion_matrix, axis=1)
        
    if np.any(correct == 0):
        return 0   # to avoid nan.
    else:
        return np.min(correct / (rec_dem +  prec_dem))

def print_metrics(x, y, classifier):
    cm = build_confusion_matrix(classifier.predict(x), y, num_classes=len(np.unique(y)))
    acc = accuracy(cm)
    prec = precision(cm)
    rec = recall(cm)
    f1 = f1_score(cm)
    
    print('Accuracy: {:.2f}. (Min-class) Precision: {:.2f}. (Min-class) Recall: {:.2f}. (Min-class) F1-Score: {:.2f}. '.format(acc, prec, rec, f1))
    print('Confusion Matrix: \n', cm)


    

np.random.seed(1)
iris = datasets.load_iris()
X, y = iris.data[:,:2], iris.target
mean = X.mean(axis=0)
std = X.std(axis=0)
X = (X - mean) / std

def multiclass(strategy, classifier):
    if classifier == 'perceptron':
        base_classifier = SGDClassifier(loss='perceptron', alpha=0.001, random_state=0)
    elif classifier == 'svm':
        base_classifier = LinearSVC(random_state=0)
    
    if strategy == 'OvO':
        classifier =  OneVsOneClassifier(base_classifier)
        colors = ['g', 'r', 'b']
    elif strategy == 'OvR' or strategy == 'OvA':
        classifier =  OneVsRestClassifier(base_classifier)
        colors = ['b', 'g', 'r']
        

    classifier.fit(X, y)
    X0, X1 = X[:, 0], X[:, 1]
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    h = .02
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

    fig = plt.subplot(111)
    plot_helpers.plot_contours(fig, classifier, xx, yy, cmap=plt.cm.jet, alpha=0.3)
    
    opt = {'marker': 'bs', 'label': '-', 'x_label': '$x$', 'y_label': '$y$', 'size': 8}
    plot_helpers.plot_data(X[np.where(y == 0)[0], 0], X[np.where(y == 0)[0], 1], fig=fig, options=opt)
    opt = {'marker': 'g*', 'label': '+', 'size': 8}
    plot_helpers.plot_data(X[np.where(y == 1)[0], 0], X[np.where(y == 1)[0], 1], fig=fig, options=opt)
    opt = {'marker': 'ro', 'label': '-', 'x_label': '$x$', 'y_label': '$y$', 'size': 8, 'legend': True}
    plot_helpers.plot_data(X[np.where(y == 2)[0], 0], X[np.where(y == 2)[0], 1], fig=fig, options=opt)
    

    def plot_hyperplane(clf, color):
        coef = clf.coef_
        intercept = clf.intercept_
        def line(x0):
            return (-(x0 * coef[0, 0]) - intercept[0]) / coef[0, 1]

        plt.plot([x_min, x_max], [line(x_min), line(x_max)],
                 ls="--", color=color)

    
    for estimator, color in zip(classifier.estimators_, colors):
        plot_hyperplane(estimator, color)

    fig.set_xlim([x_min, x_max])
    fig.set_ylim([y_min, y_max]);
    
    print_metrics(X, y, classifier)

interact(multiclass, strategy=['OvO', 'OvA'], classifier=['svm', 'perceptron']); 